In [ ]:
import pysr

In [ ]:
import os
import sys
workdir = os.getenv("PATH_TO_PARTGP")
sys.path.append(workdir)

import torch
import uproot
import awkward as ak
import numpy
import matplotlib.pyplot as plt
import vector
from pysr import PySRRegressor
import pandas as pd

from weaver.utils.dataset import SimpleIterDataset
from utils.nn_utils.part_prediction import test_load

In [ ]:
path_to_outputs = workdir + '/outputs/part_outputs/model_preds/demo_outputs.root'
path_to_config = workdir + '/data_config/JetClass/JetClass_full.yaml'

In [ ]:
rootdir = uproot.open(path_to_outputs)

In [ ]:
preds = rootdir['predictions']
houtputs = rootdir['hook_outputs']

In [ ]:
houtputs['logits'].array()

In [ ]:
preds.show()

In [ ]:
def calc_mass(pt, phi, eta, e):
    """
    Calculate Invariant Mass
    """
    invariant_mass = vector.zip({'pt': pt, 'phi': phi, 'eta': eta, 'E': e}).M
    return invariant_mass

In [ ]:
'''
We will be using observers for our inputs for a toy example.
'''

filter_out = ['scores', 'labels__label_']
features = [key for key in preds.keys() if key not in filter_out]

In [ ]:
sample_data = []

for data in preds.iterate(features, library = 'ak'):

    data['mass'] = calc_mass(data['jet_pt'], data['jet_phi'], data['jet_eta'], data['jet_energy'])
    sample_data.append(data)

all_data = ak.concatenate(sample_data)

In [ ]:
df = pd.DataFrame.from_records(all_data.to_list())

inputs = df.values
input_names = df.columns.tolist()
outputs = houtputs['logits'].array(library = 'np')

In [ ]:
sr_outputs = workdir + '/outputs/pysr_outputs'

In [ ]:
model = PySRRegressor(
    maxsize=40,
    niterations=50,
    populations=31,
    population_size = 20,
    ncycles_per_iteration = 100,
    binary_operators=["+", "-", "*", "/", "^"],
    unary_operators = ["abs", "log", "sqrt", "relu"],
    constraints = {'^': (-1, 1)},
    output_directory = sr_outputs
    elementwise_loss="loss(prediction, target) = (prediction - target)^2")

In [ ]:
model.fit(inputs, outputs, variable_names = input_names)

In [ ]:
import glob

file_list = glob.glob(sr_outputs + '/*.csv')

In [ ]:
file_list

In [ ]:
test = pd.read_csv(file_list[0])

In [ ]:
test

In [ ]:
test['Equation'][11]